# Constrained Eddy Current Problem in 2D with the Magnetic Vector Potential


\begin{align*}
\int_\Omega \nu \nabla u \cdot \nabla u'  + j\omega\sigma u u' \;d\Omega = 0
\end{align*}
with $u = B_0 x$ or $u = B_0 y$ on the boundary $\Gamma_D = \Gamma_{left}\cup\Gamma_{right}$ or $\Gamma_D = \Gamma_{top}\cup\Gamma_{bottom}$

and the constraint for each sheet:
\begin{align*}
\int_{\Omega_{c, i} } J \;d\Omega = 0
\end{align*}


results in an extension of the base with lagrange multipliers

\begin{align*}
\int_\Omega \nu \nabla u \cdot \nabla u'  + j\omega\sigma (u+\lambda_i)(u'+\lambda_i') \;d\Omega = 0
\end{align*}

In [3]:
from netgen.occ import *
import numpy as np
from ngsolve import *
from ngsolve.webgui import Draw
import cempy as cp

maxh = 1/10



freq = 50
mu0 = 4e-7*np.pi

mu_Air = mu0

sigma_Fe =2e6


omega = freq*2*np.pi

delta = np.sqrt(2/(sigma_Fe*omega*1000*mu_Air))


In [4]:
Nsheets = 6
ff = 0.9
d = delta/2



order0 = 2

B0 = 1

In [5]:
dFe = d*ff
d0 = d*(1-ff)

H_core = Nsheets*dFe + (Nsheets-1)*d0
W_core = H_core

W = 2*W_core
H = 2*H_core

wp = WorkPlane()
outer = wp.RectangleC(W, H).Face()
outer.name = "air"
outer.edges.Max(X).name = "right"
outer.edges.Min(X).name = "left"
outer.edges.Max(Y).name = "top"
outer.edges.Min(Y).name = "bottom"


rec_sheets =[]
x_pos = - W_core/2


for i in range(Nsheets):
    wp.MoveTo(x_pos, -H_core/2)

    rec_sheets.append(wp.Rectangle(dFe, H_core).Face())
    rec_sheets[-1].name = f"iron{i}"


    x_pos += d

print(x_pos, W_core/2)

rec_sheets = Glue(rec_sheets)
rec_sheets.edges.maxh = delta/10

geo = Glue([outer - rec_sheets, rec_sheets])



meshRef = Mesh(OCCGeometry(geo, dim=2).GenerateMesh(maxh=delta))
Draw(meshRef.MaterialCF({"iron.*":1, "air":2}, default=0), meshRef)
# Draw(x, meshRef)


print("domains", set(meshRef.GetMaterials()))
print("bnds", set(meshRef.GetBoundaries()))

print("penetration depth", delta)

0.002427112882151404 0.0023475354106054563


WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

domains {'iron3', 'iron5', 'iron1', 'iron2', 'iron4', 'iron0', 'air'}
bnds {'bottom', 'default', 'left', 'top', 'right'}
penetration depth 0.0015915494309189533


In [6]:
excitation_orientation = "y"

# ------------------------------------------------------------------------------
# --- Excitation
# ------------------------------------------------------------------------------
H0_amp = 1

# ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
# +++ reference solution
# ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
print("order0", order0)
print("numSheets", Nsheets)

if excitation_orientation == "x":
    dir_A = "top|bottom"
else:
    dir_A = "left|right"
VA = H1(meshRef,order=order0, complex=False, dirichlet=dir_A)
VNum = []

for i in range(Nsheets):
    VNum.append(NumberSpace(meshRef, complex=False, definedon=meshRef.Materials(f"iron{i}")))


V = FESpace([VA] + VNum)
ndof = V.ndof	
print(f"VA  :{VA.ndof}")    
print(f"ndof  :{ndof}")    

# Gridfunctions
sol_ref = GridFunction(V, "sol") 
sol_ref_o = GridFunction(V, "sol") 
A_ref = sol_ref.components[0] 
A_ref_o = sol_ref_o.components[0] 


trials = V.TrialFunction()
tests  = V.TestFunction()

uA = trials[0]
vA = tests[0]

# ------------------------------------------------------------------------------
# --- field quantities
# ------------------------------------------------------------------------------
gradA = grad(A_ref)
B = CF((gradA[1], -gradA[0])) 

E = -1j*omega*(sum(sol_ref.components))


order0 2
numSheets 6
VA  :5425
ndof  :5431


In [7]:
ev = cp.preisachScalar.Everett_Lorentzian(401, 1640, 1.5)

filename = "save/EverettMatrix_inverse"+  ev.name + "_" + str(ev.NA)  + ("_nonlin" if ev.isNonLin else "")

iev = ev.copy(False)

if not iev.Load(filename + "_2D.bin"):
    
    if not iev.Load(filename + "_1D.bin"):
        p = cp.preisachScalar.preisach(ev)
        print("calculate 1D inverse")
        iev = p.getInverseEverettMatrix()

        iev.Save(filename + "_1D.bin")
    else:
        print("loaded 1D")


    print("calculate 2D inverse")
    input()
    iev.Generate2DAdaption(4)
    iev.Save(filename + "_2D.bin")
else:
    print("loaded 2D")

loaded 2D


In [8]:
intrule = IntegrationRule(TRIG, order0+2)
dist = cp.preisachVector.circleDistribution(40)
Binput = B.real


mask = meshRef.MaterialCF({"iron.*":1}, default=0)

intrule = IntegrationRule(TRIG, 2*order0+2)
dist = cp.preisachVector.circleDistribution(3)
Binput = B.real





In [10]:
Preisach_ref = cp.preisachVector.ngPreisachVector2(meshRef, intrule, Binput, iev, dist, field="B", mask = mask )
print(Preisach_ref.CountVectorPreisachPoints())

17748


In [11]:
nu_preisach = Preisach_ref.GetNu()
H_preisach = Preisach_ref.GetH()

nu = meshRef.MaterialCF({"iron.*":nu_preisach, "air":1/mu_Air}, default=0)
sigma = meshRef.MaterialCF({"iron.*":sigma_Fe}, default=0)



rho = 1/sigma


H = meshRef.MaterialCF({"iron.*":H_preisach, "air":1/mu_Air  * B}, default=0)
J = sigma * E

In [12]:
ti = np.linspace(0, 1/freq*1.25, 200)
dt = ti[1] - ti[0]
Bin = B0*np.sin(ti)




In [13]:
fesFP = MatrixValued(L2(meshRef, definedon=meshRef.Materials("iron.*")))
nu_FP = GridFunction(fesFP)
nu_FP_init = 1/(700 * mu0)
nu_FP.Set(CF((nu_FP_init, 0, 0, nu_FP_init), dims=(2,2)))

In [14]:



# ------------------------------------------------------------------------------
# Matrix
# ------------------------------------------------------------------------------
with TaskManager():
    # Bilinear form with 
    ah_ref = BilinearForm(V, symmetric=True)

    # A:
    ah_ref += dt * nu_FP *grad(uA) * grad(vA) * dx

    # lagrange multipliers
    for i in range(Nsheets):
        ah_ref += sigma_Fe * (uA + trials[1+i]) * (vA + tests[1+i]) * dx(f"iron{i}")


    prec = Preconditioner(ah_ref, type="direct")
    ah_ref.Assemble()

print(ah_ref.mat.AsVector().Norm())

f_ref = LinearForm(V) 


# lagrange multipliers
for i in range(Nsheets):
    # Euler
    f_ref += sigma_Fe * A_ref * vA * dx(f"iron{i}")

    # nonlin
    gradvA = grad(vA)
    curlvA = CF((gradA[1], -gradA[0]))
    f_ref += dt * (nu_FP * grad(A_ref)) * grad(vA) * dx(f"iron{i}")
    f_ref += dt * (- H_preisach)* curlvA * dx(f"iron{i}")









22.28743100762064


In [15]:
# ------------------------------------------------------------------------------
# ------ Solve It
# ------------------------------------------------------------------------------
with TaskManager():
    

    for i in range(len(Bin)):
        print("-------------", i, len(Bin)-1)
        sol_ref_o.vec.data = sol_ref.vec

        if i> 0:
            nu_FP.Set(nu_preisach)

            ah_ref.Assemble()


        if excitation_orientation == "x":
            A_ref.Set(Bin[i]*y, BND)
        else:
            A_ref.Set(Bin[i]*x, BND)

        for it in range(20):
            
            # solve it
            f_ref.Assemble()
            solvers.BVP(bf=ah_ref, lf=f_ref, gf=sol_ref, pre=prec, maxsteps=5)

            Preisach_ref.Update()

        print("doen")

        Preisach_ref.UpdatePast()
            





        Draw(J.imag, meshRef)
        Draw(B.real, meshRef, vectors=True)

    

------------- 0 199
0
CG iteration 1, residual = 0.0     
1
CG iteration 1, residual = 0.0     
2
CG iteration 1, residual = 0.0     
3
CG iteration 1, residual = 0.0     
4
CG iteration 1, residual = 0.0     
5
CG iteration 1, residual = 0.0     
6
CG iteration 1, residual = 0.0     
7
CG iteration 1, residual = 0.0     
8
CG iteration 1, residual = 0.0     
9
CG iteration 1, residual = 0.0     
10
CG iteration 1, residual = 0.0     
11
CG iteration 1, residual = 0.0     
12
CG iteration 1, residual = 0.0     
13
CG iteration 1, residual = 0.0     
14
CG iteration 1, residual = 0.0     
15
CG iteration 1, residual = 0.0     
16
CG iteration 1, residual = 0.0     
17
CG iteration 1, residual = 0.0     
18


: 